# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from pprint import pprint
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statistics
from google.colab import drive
drive.mount('/content/drive/')
from sklearn.tree import DecisionTreeClassifier
from category_encoders import TargetEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#pip install category_encoders

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
class Node:
  def __init__(self):
    self.R = None
    self.L = None
    self.classP = None
    self.col = None
    self.value = -1
    self.leaf = False


class DT:
  def __init__(self, depth, minS):
    self.tree = None
    self.depth = depth
    self.nowD = 0
    self.minS = minS
    self.y_train = None
  def findClass(self, x_test, node):
    if node.leaf:
      return node.classP
    elif x_test[node.col] > node.value:
      return self.findClass(x_test, node.R)
    else:
      return self.findClass(x_test, node.L)
  def fit(self, x_train, y_train):
    self.y_train = y_train
    self.tree = Node()
    self.tree.classP = self.calcClassP(y_train)
    self.SplitNode(np.asarray(x_train), y_train, self.tree)
  def predict(self, x_test):
    y_pred = np.array([])
    for x in np.asarray(x_test):
      y_pred = np.append(y_pred, np.argmax(self.findClass(x, self.tree)))
    return y_pred
  def SplitNode(self, x_train, y_train, node):
    if self.nowD >= self.depth or len(np.unique(y_train)) == 1:
      node.leaf = True
      return
    bestFeature = None
    bestT = None
    bestEntropy = -1
    bestSampleL = []
    bestSampleR = []
    xL = []
    xR = []
    for feature in range(x_train.shape[1]):
      xFeature = x_train[:, feature]
      for value in xFeature:
        sampleL = y_train[xFeature <= value]
        sampleR = y_train[xFeature > value]
        if len(sampleL) == 0 or len(sampleR) == 0:
          continue
        entropyVal = entropy(y_train) - (
            (len(sampleL) / len(y_train) * entropy(sampleL)) + (
                len(sampleR) / len(y_train)*entropy(sampleR)))
        if entropyVal > bestEntropy:
          bestT = value
          bestEntropy = entropyVal
          bestSampleL = sampleL
          bestSampleR = sampleR
          bestCol = feature
          xL = x_train[xFeature <= value, :]
          xR = x_train[xFeature > value, :]
          xFeature = x_train[:, bestCol]
      if len(xL) < self.minS or len(xR) < self.minS:
        node.leaf = True
        return
    self.nowD += 1
    node.L = Node()
    node.L.classP = self.calcClassP(bestSampleL)
    node.R = Node()
    node.R.classP = self.calcClassP(bestSampleR)
    node.col = bestCol
    node.value = bestT
    self.SplitNode(xL, bestSampleL, node.L)
    self.SplitNode(xR, bestSampleR, node.R)
  def calcClassP(self, data):
    result = np.array([])
    for type_ in np.unique(self.y_train):
      result = np.append(result, len(data[data == type_]) / len(data))
    return result

In [ ]:
class DT1:
  def __init__(self, depth, minS):
    self.tree = None
    self.depth = depth
    self.nowD = 0
    self.minS = minS
    self.y_train = None
  def findClass(self, x_test, node):
    if node.leaf:
      return node.classP
    elif x_test[node.col] > node.value:
      return self.findClass(x_test, node.R)
    else:
      return self.findClass(x_test, node.L)
  def fit(self, x_train, y_train):
    self.y_train = y_train
    self.tree = Node()
    self.tree.classP = self.calcClassP(y_train)
    self.SplitNode(np.asarray(x_train), y_train, self.tree)
  def predict(self, x_test):
    y_pred = np.array([])
    for x in np.asarray(x_test):
      y_pred = np.append(y_pred, np.argmax(self.findClass(x, self.tree)))
    return y_pred
  def SplitNode(self, x_train, y_train, node):
    if self.nowD >= self.depth or len(np.unique(y_train)) == 1:
      node.leaf = True
      return
    bestFeature = None
    bestT = None
    bestEntropy = -1
    bestSampleL = []
    bestSampleR = []
    xL = []
    xR = []
    for feature in range(x_train.shape[1]):
      xFeature = x_train[:, feature]
      for value in xFeature:
        sampleL = y_train[xFeature <= value]
        sampleR = y_train[xFeature > value]
        if len(sampleL) == 0 or len(sampleR) == 0:
          continue
        entropyVal = entropy(y_train) - (
            (len(sampleL) / len(y_train) * entropy(sampleL)) + (
                len(sampleR) / len(y_train)*entropy(sampleR)))
        if entropyVal > bestEntropy:
          bestT = value
          bestEntropy = entropyVal
          bestSampleL = sampleL
          bestSampleR = sampleR
          bestCol = feature
          xL = x_train[xFeature <= value, :]
          xR = x_train[xFeature > value, :]
          xFeature = x_train[:, bestCol]
      if len(xL) < self.minS or len(xR) < self.minS:
        node.leaf = True
        return
    self.nowD += 1
    node.L = Node()
    node.L.classP = self.calcClassP(bestSampleL)
    node.R = Node()
    node.R.classP = self.calcClassP(bestSampleR)
    node.col = bestCol
    node.value = bestT
    self.SplitNode(xL, bestSampleL, node.L)
    self.SplitNode(xR, bestSampleR, node.R)
  def calcClassP(self, data):
    result = np.array([])
    for type_ in np.unique(self.y_train):
      result = np.append(result, len(data[data == type_]) / len(data))
    return result

In [ ]:
class DT2:
  def __init__(self, depth, minS):
    self.tree = None
    self.depth = depth
    self.nowD = 0
    self.minS = minS
    self.y_train = None
  def findClass(self, x_test, node):
    if node.leaf:
      return node.classP
    elif x_test[node.col] > node.value:
      return self.findClass(x_test, node.R)
    else:
      return self.findClass(x_test, node.L)
  def fit(self, x_train, y_train):
    self.y_train = y_train
    self.tree = Node()
    self.tree.classP = self.calcClassP(y_train)
    self.SplitNode(np.asarray(x_train), y_train, self.tree)
  def predict(self, x_test):
    y_pred = np.array([])
    for x in np.asarray(x_test):
      y_pred = np.append(y_pred, np.argmax(self.findClass(x, self.tree)))
    return y_pred
  def SplitNode(self, x_train, y_train, node):
    if self.nowD >= self.depth or len(np.unique(y_train)) == 1:
      node.leaf = True
      return
    bestFeature = None
    bestT = None
    bestEntropy = -1
    bestSampleL = []
    bestSampleR = []
    xL = []
    xR = []
    for feature in range(x_train.shape[1]):
      xFeature = x_train[:, feature]
      for value in xFeature:
        sampleL = y_train[xFeature <= value]
        sampleR = y_train[xFeature > value]
        if len(sampleL) == 0 or len(sampleR) == 0:
          continue
        entropyVal = entropy(y_train) - (
            (len(sampleL) / len(y_train) * entropy(sampleL)) + (
                len(sampleR) / len(y_train)*entropy(sampleR)))
        if entropyVal > bestEntropy:
          bestT = value
          bestEntropy = entropyVal
          bestSampleL = sampleL
          bestSampleR = sampleR
          bestCol = feature
          xL = x_train[xFeature <= value, :]
          xR = x_train[xFeature > value, :]
          xFeature = x_train[:, bestCol]
      if len(xL) < self.minS or len(xR) < self.minS:
        node.leaf = True
        return
    self.nowD += 1
    node.L = Node()
    node.L.classP = self.calcClassP(bestSampleL)
    node.R = Node()
    node.R.classP = self.calcClassP(bestSampleR)
    node.col = bestCol
    node.value = bestT
    self.SplitNode(xL, bestSampleL, node.L)
    self.SplitNode(xR, bestSampleR, node.R)
  def calcClassP(self, data):
    result = np.array([])
    for type_ in np.unique(self.y_train):
      result = np.append(result, len(data[data == type_]) / len(data))
    return result

In [ ]:
class DT3:
  def __init__(self, depth, minS):
    self.tree = None
    self.depth = depth
    self.nowD = 0
    self.minS = minS
    self.y_train = None
  def findClass(self, x_test, node):
    if node.leaf:
      return node.classP
    elif x_test[node.col] > node.value:
      return self.findClass(x_test, node.R)
    else:
      return self.findClass(x_test, node.L)
  def fit(self, x_train, y_train):
    self.y_train = y_train
    self.tree = Node()
    self.tree.classP = self.calcClassP(y_train)
    self.SplitNode(np.asarray(x_train), y_train, self.tree)
  def predict(self, x_test):
    y_pred = np.array([])
    for x in np.asarray(x_test):
      y_pred = np.append(y_pred, np.argmax(self.findClass(x, self.tree)))
    return y_pred
  def SplitNode(self, x_train, y_train, node):
    if self.nowD >= self.depth or len(np.unique(y_train)) == 1:
      node.leaf = True
      return
    bestFeature = None
    bestT = None
    bestEntropy = -1
    bestSampleL = []
    bestSampleR = []
    xL = []
    xR = []
    for feature in range(x_train.shape[1]):
      xFeature = x_train[:, feature]
      for value in xFeature:
        sampleL = y_train[xFeature <= value]
        sampleR = y_train[xFeature > value]
        if len(sampleL) == 0 or len(sampleR) == 0:
          continue
        entropyVal = entropy(y_train) - (
            (len(sampleL) / len(y_train) * entropy(sampleL)) + (
                len(sampleR) / len(y_train)*entropy(sampleR)))
        if entropyVal > bestEntropy:
          bestT = value
          bestEntropy = entropyVal
          bestSampleL = sampleL
          bestSampleR = sampleR
          bestCol = feature
          xL = x_train[xFeature <= value, :]
          xR = x_train[xFeature > value, :]
          xFeature = x_train[:, bestCol]
      if len(xL) < self.minS or len(xR) < self.minS:
        node.leaf = True
        return
    self.nowD += 1
    node.L = Node()
    node.L.classP = self.calcClassP(bestSampleL)
    node.R = Node()
    node.R.classP = self.calcClassP(bestSampleR)
    node.col = bestCol
    node.value = bestT
    self.SplitNode(xL, bestSampleL, node.L)
    self.SplitNode(xR, bestSampleR, node.R)
  def calcClassP(self, data):
    result = np.array([])
    for type_ in np.unique(self.y_train):
      result = np.append(result, len(data[data == type_]) / len(data))
    return result

In [ ]:
iris = datasets.load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.3)
dt = DT(5, 5)
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
print(f'accuracy score = {accuracy_score(y_test, y_pred)}')

accuracy score = 0.9555555555555556


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
class RF:
  def __init__(self, tree_count, depth, min_leaves):
    self.tree_count = tree_count
    self.depth = depth
    self.min_leaves = min_leaves
    self.trees = []
  def fit(self, x_train, y_train):
    features = int(np.sqrt(x_train.shape[1]))
    for i in range(self.tree_count):
      tree = DecisionTreeClassifier(max_features = features, 
                                    max_depth = self.depth,
                                    min_samples_leaf = self.min_leaves)
      tree.fit(x_train, y_train)
      self.trees.append(tree)
  def predict(self, x_test):
    result = []
    for tree in self.trees:
      result.append(tree.predict(x_test))
    return round(pd.DataFrame(data = result).mean())
  def importances(self):
    result = []
    for tree in self.trees:
      result.append(tree.feature_importances_)
    return pd.DataFrame(result).mean()

In [ ]:
data = pd.read_csv(r"/content/drive/MyDrive/ML/churn.csv")
target = data['Exited']
data['Gender'] = TargetEncoder().fit_transform(data['Gender'], target)
data['Geography'] = TargetEncoder().fit_transform(data['Geography'], target)
data = data.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis = 1)
names = data.columns
scaler = MinMaxScaler()
d = scaler.fit_transform(data)
scaled_data = pd.DataFrame(d, columns = names)
scaled_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0.538,0.00000,1.0,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735
1,0.516,0.03184,1.0,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709
2,0.304,0.00000,1.0,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654
3,0.698,0.00000,1.0,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120
4,1.000,0.03184,1.0,0.337838,0.2,0.500246,0.000000,1.0,1.0,0.395400
...,...,...,...,...,...,...,...,...,...,...
9995,0.842,0.00000,0.0,0.283784,0.5,0.000000,0.333333,1.0,0.0,0.481341
9996,0.332,0.00000,0.0,0.229730,1.0,0.228657,0.000000,1.0,1.0,0.508490
9997,0.718,0.00000,1.0,0.243243,0.7,0.000000,0.000000,0.0,1.0,0.210390
9998,0.844,1.00000,0.0,0.324324,0.3,0.299226,0.333333,1.0,0.0,0.464429


In [15]:
x_train, x_test, y_train, y_test = train_test_split(scaled_data, target, test_size = 0.3)
bestDepth, bestNum, bestMinLeaves, bestScore = 0, 0, 0, 0
for num in range(100, 500, 10):
  for depth in range(1, 5):
    for MinLeaves in range(1, 5):
      model = RF(num, depth, MinLeaves)
      model.fit(x_train, y_train)
      y_pred = model.predict(x_test)
      score = accuracy_score(y_test, y_pred)
      if score > bestScore:
        print(score)
        #если новый лучший результат, смотрим скор на тесте
        bestDepth, bestNum, bestMinLeaves, bestScore = depth, num, MinLeaves, score
print(f'Best:\ndepth {bestDepth}\nNum {bestNum}\nMinLeaves {bestMinLeaves}')
model = RF(bestNum, bestDepth, bestMinLeaves)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(f'final score :{accuracy_score(y_test, y_pred)}')

0.7926666666666666


KeyboardInterrupt: ignored

In [ ]:
model = RF(230, 4, 2)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(f'final score :{accuracy_score(y_test, y_pred)}')
print(pd.DataFrame(data=np.c_[x_test.columns, model.importances()], columns=["features", "pow"]).sort_values(by="pow", ascending=False))

final score :0.848
          features          pow
3              Age     0.418179
6    NumOfProducts     0.337203
8   IsActiveMember     0.108735
1        Geography    0.0741852
5          Balance     0.034692
2           Gender    0.0127825
0      CreditScore   0.00992237
9  EstimatedSalary   0.00255362
4           Tenure   0.00135903
7        HasCrCard  0.000388575


In [ ]:
forest = RandomForestClassifier()
forest.fit(x_train, y_train)
y_pred = forest.predict(x_test)
print(f'{accuracy_score(y_test, y_pred)}')
print(pd.DataFrame(data=np.c_[x_test.columns, forest.feature_importances_], columns=["features", "pow"]).sort_values(by="pow", ascending=False))

0.858
          features        pow
3              Age   0.232652
5          Balance   0.145239
9  EstimatedSalary    0.14354
0      CreditScore    0.14092
6    NumOfProducts   0.134013
4           Tenure   0.078012
8   IsActiveMember   0.043998
1        Geography  0.0429001
2           Gender  0.0198844
7        HasCrCard  0.0188419
